# Data Mining : Midterm Project

Name: <b>Mihir Kadam</b><br>
Ucid: mk2436<br>
Email Id: mk2436@njit.edu<br>
<br>
Submitted to: <b>Dr. Yasser Abduallah</b><br>
<br>

### Description

This notebook implements and compares three frequent itemset mining algorithms: **Brute Force**, **Apriori**, and **FP-Growth**. It allows users to load transaction data from CSV files, set minimum support and confidence thresholds, and analyze the performance and results of each algorithm in finding frequent itemsets and generating association rules.

### Algorithms Implemented

- **Brute Force**: A naive approach that checks all possible item combinations.
- **Apriori**: A classic algorithm that uses the anti-monotonic property of support to efficiently find frequent itemsets.
- **FP-Growth**: An algorithm that uses a frequent pattern tree (FP-tree) to mine frequent itemsets without generating candidate itemsets.


### Usage

1. Upload your transaction and itemset CSV files to the Colab environment. Example files are provided for several stores (Amazon, Bestbuy, Shoprite, K-mart, Nike).
2. Run all the code cells.
3. When prompted, select the store you want to analyze by entering its corresponding number.
4. Enter the minimum support and confidence thresholds as percentages (between 1 and 100).
5. The notebook will then run each algorithm, display the frequent itemsets and association rules found, and show the execution time for each.
6. Finally, it will identify and print the fastest algorithm based on execution time.

### Data Format

- **Transaction File**: A CSV file where each row represents a transaction. Items within a transaction can be in separate columns or comma-separated within a single cell.
- **Itemset File**: A CSV file with at least two columns: 'Item #' (item ID) and 'Item Name' (item name). This file is used to map item IDs to names if available.

### Dependencies

- pandas
- mlxtend
- tabulate

Install dependencies using pip if necessary:
`!pip install pandas mlxtend tabulate`

### Notes

- The Brute Force algorithm can be very slow for large datasets due to its exhaustive search approach.
- Apriori and FP-Growth algorithms are more efficient for larger datasets.
- The results are printed in a tabular format for easy comparison.

In [ ]:
import warnings

# If you see the warning:
# "DeprecationWarning: datetime.datetime.utcnow() is deprecated ..."
# run this cell before executing any other code to prevent the warning
# from breaking your notebook.

warnings.filterwarnings(
    "ignore",
    category=DeprecationWarning,
    message=r"datetime\.datetime\.utcnow\(\).*deprecated",
)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [ ]:
import itertools
import time
import pandas as pd
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules
from tabulate import tabulate

### How `calculate_support` works:

This function helps us understand how popular a specific group of items (an 'itemset') is by looking at all the customer transactions. It figures out what percentage of all transactions include every single item in the itemset you're interested in.

Think of it like this: If you want to know the support for 'milk and bread', this function counts how many transactions have *both* milk and bread, and then divides that by the total number of transactions. The result, a number between 0 and 1, tells you how frequently 'milk and bread' appear together.

In [ ]:
def calculate_support(itemset, transactions):
    """
    Return the fraction of transactions containing the itemset.

    Args:
        itemset (tuple or list): Items to check.
        transactions (list of lists): List of transactions.

    Returns:
        float: Support value (0 to 1).
    """
    try:
        return sum(1 for t in transactions if set(itemset).issubset(set(t))) / len(transactions)
    except Exception as e:
        print(f"Error calculating support \n{e}")


### How `prepare_transaction_dataframe` works:

This function takes your list of customer transactions and turns it into a format that's easy for algorithms like Apriori and FP-Growth to understand. It creates a table (a pandas DataFrame) where:

- Each row represents a single transaction.
- Each column represents a unique item found across all transactions.

For each cell in this table, it simply puts a `True` if the item in that column was present in the transaction for that row, and `False` otherwise. This "one-hot encoding" makes it straightforward for itemset mining algorithms to quickly see which items appear together in transactions.

In [ ]:
def prepare_transaction_dataframe(transactions):
    """
    Convert a list of transactions into a one-hot encoded pandas DataFrame.

    Each column represents an item, and each row represents a transaction.
    True indicates the item is present in the transaction; False otherwise.

    Args:
        transactions (list of lists): List of transactions, where each transaction is a list of items.

    Returns:
        pandas.DataFrame: One-hot encoded DataFrame of shape (num_transactions, num_items).
    """
    try:
        items = sorted(set(i for t in transactions for i in t))
        df = pd.DataFrame([[i in t for i in items] for t in transactions], columns=items).astype(bool)
        return df
    except Exception as e:
        print(f"Error loading dataframe \n{e}")


### How `load_transaction_data` works:

This function is responsible for reading your raw transaction and item information from CSV files. It does two main things:

1.  **Reads Transactions:** It opens the transaction CSV and goes through each row. It expects items within a row to be either in separate columns or separated by commas. It collects all the items from each valid row to form a list of transactions.
2.  **Creates Item Map (if available):** If you provide an itemset CSV with 'Item #' and 'Item Name' columns, this function creates a dictionary that links the item's ID (number) to its actual name. This is helpful for displaying results with understandable item names instead of just numbers.

Essentially, it prepares your raw data into a structured format (a list of transactions and an optional item name map) that the other functions in the notebook can then process.

In [ ]:
def load_transaction_data(transaction_file, itemset_file):
    """
    Load transaction and itemset data from CSV files.

    Reads a transaction CSV file and an itemset CSV file. Processes the transactions
    into a list of lists, where each inner list represents a single transaction. If the
    itemset file contains 'Item #' and 'Item Name' columns, a mapping dictionary is created.

    Args:
        transaction_file (str): Path to the CSV file containing transaction data.
        itemset_file (str): Path to the CSV file containing item ID and item name mapping.

    Returns:
        tuple:
            - transactions (list of lists): Each inner list represents a transaction with item names as strings.
            - item_map (dict or None): Mapping from item IDs to item names if available, otherwise None.

    Notes:
        - Items in transaction rows can be comma-separated strings or numeric values.
        - Empty or invalid transactions are ignored.
    """
    try:
        df_trans = pd.read_csv(transaction_file)
        df_items = pd.read_csv(itemset_file)

        if 'Item #' in df_items.columns and 'Item Name' in df_items.columns:
            item_map = dict(zip(df_items['Item #'], df_items['Item Name']))
        else:
            item_map = None

        transactions = []
        for _, row in df_trans.iterrows():
            transaction = []
            for item in row:
                if isinstance(item, str):
                    items = [i.strip() for i in item.split(',') if i.strip()]
                    transaction.extend(items)
                elif pd.notna(item):
                    transaction.append(str(item))
            if transaction:
                transactions.append(transaction)
        return transactions, item_map

    except Exception as e:
        print(f"Error reading CSVs \n{e}")
        return None, None


### How `brute_force_algorithm` works:

This function implements the Brute Force approach to find frequent itemsets and association rules. It works by:

1.  **Generating Candidates:** It creates every possible combination of items from the transactions, starting with single items and going up to combinations of all items.
2.  **Calculating Support:** For each combination (itemset), it checks how many transactions contain all the items in that set and calculates the support (the percentage of transactions containing the itemset).
3.  **Filtering Frequent Itemsets:** It keeps only those itemsets whose support is equal to or greater than the minimum support threshold you specified.
4.  **Generating Association Rules:** From the frequent itemsets, it creates possible association rules (e.g., "if a customer buys A, they also buy B").
5.  **Calculating Confidence:** For each rule, it calculates the confidence (how often itemset B appears in transactions that already contain itemset A).
6.  **Filtering Rules:** It keeps only those rules whose confidence is equal to or greater than the minimum confidence threshold you specified.

Because it checks every single possible combination, this method can be very slow for large datasets.

In [ ]:
def brute_force_algorithm(transactions, min_support, min_confidence):
    """
    Run the Brute Force algorithm to find frequent itemsets and generate association rules.

    This function exhaustively generates all possible item combinations from the transactions,
    calculates their support, and retains those meeting the minimum support threshold.
    It then generates association rules from frequent itemsets, keeping only those with
    confidence above the specified threshold.

    Args:
        transactions (list of lists): A list where each inner list represents a transaction containing items as strings.
        min_support (float): Minimum support threshold (0 <= min_support <= 1) for an itemset to be considered frequent.
        min_confidence (float): Minimum confidence threshold (0 <= min_confidence <= 1) for association rules.

    Returns:
        tuple:
            - frequent_itemsets (list of tuples): Each tuple contains (itemset, support) for itemsets meeting the min_support.
            - rules (list of tuples): Each tuple contains (antecedent, consequent, confidence, support) for rules meeting min_confidence.
            - runtime (float): Execution time in seconds for the algorithm.

    Notes:
        - This is a computationally expensive method for large datasets, as it checks all possible item combinations.
        - If no frequent itemsets are found, returns (None, None, runtime).
        - The function prints results in a tabular format using `print_results`.
    """

    print("\nRunning Brute Force Algorithm...")
    start = time.time()
    try:
        items = sorted(set(i for t in transactions for i in t))
        frequent_itemsets = []
        size = 1

        while True:
            candidates = list(itertools.combinations(items, size))
            current = []
            for itemset in candidates:
                support = calculate_support(itemset, transactions)
                if support >= min_support:
                    current.append((itemset, support))
            if not current:
                break
            frequent_itemsets.extend(current)
            size += 1

        # Generate rules
        rules = []
        for itemset, support in frequent_itemsets:
            if len(itemset) > 1:
                for i in range(1, len(itemset)):
                    for antecedent in itertools.combinations(itemset, i):
                        consequent = tuple(x for x in itemset if x not in antecedent)
                        antecedent_support = calculate_support(antecedent, transactions)
                        confidence = support / antecedent_support
                        if confidence >= min_confidence:
                            rules.append((antecedent, consequent, confidence, support))
    except Exception as e:
        print(f"Error running Brute Force \n{e}")

    finally:
        runtime = time.time() - start

    if not frequent_itemsets:
        print("No frequent itemsets found with Brute Force.")
        return None, None, runtime

    print(f"Brute Force completed in {runtime:.4f} seconds.\n")
    print_results("Brute Force", frequent_itemsets, rules)
    return frequent_itemsets, rules, runtime


### How `apriori_algorithm` works:

This function implements the Apriori algorithm using the `mlxtend` library. It's a more efficient method than Brute Force because it uses a key property: if an itemset is frequent, all of its subsets must also be frequent. This helps prune the search space significantly. The function does the following:

1.  **Find Frequent Itemsets:** It uses the `mlxtend.apriori` function on the prepared transaction data to find all itemsets whose support meets the minimum threshold you provide. It does this by iteratively building larger frequent itemsets from smaller ones.
2.  **Generate Association Rules:** From the frequent itemsets found, it generates possible association rules (e.g., "if a customer buys A, they also buy B") using the `mlxtend.association_rules` function.
3.  **Filter Rules by Confidence:** It keeps only those rules whose confidence (how often the consequent items are purchased when the antecedent items are purchased) is at or above the minimum confidence threshold you specified.

This approach is generally much faster than Brute Force, especially for larger datasets.

In [ ]:
def apriori_algorithm(transaction_df, min_support, min_confidence):
    """
    Run the Apriori algorithm using mlxtend to find frequent itemsets and generate association rules.

    This function uses the Apriori algorithm on a one-hot encoded DataFrame of transactions to
    identify frequent itemsets that meet the minimum support threshold. It then generates
    association rules from these itemsets that meet the minimum confidence threshold.

    Args:
        transaction_df (pd.DataFrame): One-hot encoded DataFrame where each column is an item and
                                       each row represents a transaction (True if item present).
        min_support (float): Minimum support threshold (0 <= min_support <= 1) for an itemset to be considered frequent.
        min_confidence (float): Minimum confidence threshold (0 <= min_confidence <= 1) for association rules.

    Returns:
        tuple:
            - frequent_itemsets (pd.DataFrame or None): DataFrame of frequent itemsets with support values.
                                                        None if no frequent itemsets are found.
            - rules (pd.DataFrame or None): DataFrame of association rules with confidence and support.
                                            None if no rules meet the threshold.
            - runtime (float): Execution time in seconds for the algorithm.

    Notes:
        - Uses mlxtend.apriori for frequent itemset mining and mlxtend.association_rules for rule generation.
        - If no frequent itemsets are found, returns (None, None, runtime).
        - The function prints results in a tabular format using `print_results`.
    """

    print("\nRunning Apriori Algorithm...")
    frequent_itemsets = None
    start = time.time()

    try:
        frequent_itemsets = apriori(transaction_df, min_support=min_support, use_colnames=True)

    except Exception as e:
        print(f"Error running Arpiori \n{e}")

    finally:
        runtime = time.time() - start

    if frequent_itemsets.empty:
        print("No frequent itemsets found with Apriori.")
        return None, None, runtime

    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)
    print(f"Apriori completed in {runtime:.4f} seconds.\n")
    print_results("Apriori", frequent_itemsets, rules)
    return frequent_itemsets, rules, runtime


### How `fpgrowth_algorithm` works:

This function applies the FP-Growth algorithm on a one-hot encoded DataFrame of transactions
to identify frequent itemsets that meet the minimum support threshold. It then generates
association rules from these itemsets that meet the minimum confidence threshold.

Args:
transaction_df (pd.DataFrame): One-hot encoded DataFrame where each column is an item and
each row represents a transaction (True if item present).
min_support (float): Minimum support threshold (0 <= min_support <= 1) for an itemset to be considered frequent.
min_confidence (float): Minimum confidence threshold (0 <= min_confidence <= 1) for association rules.

Returns:
tuple:
- frequent_itemsets (pd.DataFrame or None): DataFrame of frequent itemsets with support values.
None if no frequent itemsets are found.
- rules (pd.DataFrame or None): DataFrame of association rules with confidence and support.
None if no rules meet the threshold.
- runtime (float): Execution time in seconds for the algorithm.

Notes:
- Uses mlxtend.fpgrowth for frequent itemset mining and mlxtend.association_rules for rule generation.
- If no frequent itemsets are found, returns (None, None, runtime).
- The function prints results in a tabular format using `print_results`.

In [ ]:
def fpgrowth_algorithm(transaction_df, min_support, min_confidence):
    """
    Run the FP-Growth algorithm using mlxtend to find frequent itemsets and generate association rules.

    This function applies the FP-Growth algorithm on a one-hot encoded DataFrame of transactions
    to identify frequent itemsets that meet the minimum support threshold. It then generates
    association rules from these itemsets that meet the minimum confidence threshold.

    Args:
        transaction_df (pd.DataFrame): One-hot encoded DataFrame where each column is an item and
                                       each row represents a transaction (True if item present).
        min_support (float): Minimum support threshold (0 <= min_support <= 1) for an itemset to be considered frequent.
        min_confidence (float): Minimum confidence threshold (0 <= min_confidence <= 1) for association rules.

    Returns:
        tuple:
            - frequent_itemsets (pd.DataFrame or None): DataFrame of frequent itemsets with support values.
                                                        None if no frequent itemsets are found.
            - rules (pd.DataFrame or None): DataFrame of association rules with confidence and support.
                                            None if no rules meet the threshold.
            - runtime (float): Execution time in seconds for the algorithm.

    Notes:
        - Uses mlxtend.fpgrowth for frequent itemset mining and mlxtend.association_rules for rule generation.
        - If no frequent itemsets are found, returns (None, None, runtime).
        - The function prints results in a tabular format using `print_results`.
    """

    print("\nRunning FP-Growth Algorithm...")
    frequent_itemsets = None
    start = time.time()

    try:

        frequent_itemsets = fpgrowth(transaction_df, min_support=min_support, use_colnames=True)

    except Exception as e:
        print(f"Error running FP-Tree \n{e}")

    finally:
        runtime = time.time() - start

    if frequent_itemsets.empty:
        print("No frequent itemsets found with FP-Growth.")
        return None, None, runtime

    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)
    print(f"FP-Growth completed in {runtime:.4f} seconds.\n")
    print_results("FP-Growth", frequent_itemsets, rules)
    return frequent_itemsets, rules, runtime


### How `print_results` works:

This function is designed to display the output of the frequent itemset mining algorithms in a clear, easy-to-read table format. It takes the results (frequent itemsets and association rules) from any of the algorithms and uses the `tabulate` library to create well-formatted tables.

Here's what it does:

1.  **Displays Algorithm Name:** It starts by printing the name of the algorithm whose results are being shown.
2.  **Formats Frequent Itemsets:** It takes the frequent itemsets found by an algorithm and puts them into a table. Each row shows an itemset and its corresponding support percentage.
3.  **Formats Association Rules:** It takes the association rules generated by an algorithm and presents them in another table. Each row shows a rule (in the format "items A -> items B"), its confidence percentage, and its support percentage.
4.  **Handles Empty Results:** If an algorithm doesn't find any frequent itemsets or association rules for the given thresholds, the function displays a message indicating that no results were found.

Essentially, this function is a utility to make the output of the mining process understandable and presentable.

In [ ]:
def print_results(algorithm_name, frequent_itemsets, rules):
    """
    Print frequent itemsets and association rules in a tabular format.

    This function takes the results from a frequent itemset mining algorithm (either as
    a pandas DataFrame or a list of tuples) and prints them neatly using `tabulate`.
    Association rules are displayed in the format "A -> B" with confidence and support.

    Args:
        algorithm_name (str): Name of the algorithm (used in the table header).
        frequent_itemsets (pd.DataFrame or list of tuples or None): Frequent itemsets with support values.
        rules (pd.DataFrame or list of tuples or None): Association rules with confidence and support.

    Returns:
        None: This function only prints the results; it does not return any value.

    Notes:
        - If no frequent itemsets or rules are found, appropriate messages are displayed.
        - Supports both DataFrame input (from mlxtend) and list-of-tuples input (from custom algorithms).
        - Association rules are displayed in the format "antecedent -> consequent".
    """

    print(f"\n=== {algorithm_name} Results ===")

    try:

        # --- Frequent Itemsets ---
        print("\nFrequent Itemsets:")
        if isinstance(frequent_itemsets, pd.DataFrame):
            itemset_data = [
                [", ".join(list(row["itemsets"])), f"{row['support']*100:.2f}%"]
                for _, row in frequent_itemsets.iterrows()
            ]
        elif frequent_itemsets:
            itemset_data = [
                [", ".join(itemset), f"{support*100:.2f}%"]
                for itemset, support in frequent_itemsets
            ]
        else:
            itemset_data = [["No frequent itemsets found", ""]]

        print(tabulate(itemset_data, headers=["Itemset", "Support"], tablefmt="fancy_grid"))

        # --- Association Rules ---
        print("\nAssociation Rules:")
        if isinstance(rules, pd.DataFrame) and not rules.empty:
            rule_data = [
                [f"{', '.join(list(rule['antecedents']))} -> {', '.join(list(rule['consequents']))}",
                f"{rule['confidence']*100:.2f}%",
                f"{rule['support']*100:.2f}%"]
                for _, rule in rules.iterrows()
            ]
        elif rules is not None and len(rules) > 0:
            rule_data = [
                [f"{', '.join(antecedent)} -> {', '.join(consequent)}",
                f"{confidence*100:.2f}%",
                f"{support*100:.2f}%"]
                for antecedent, consequent, confidence, support in rules
            ]
        else:
            rule_data = [["No association rules found", "", ""]]

        print(tabulate(rule_data, headers=["Rule", "Confidence", "Support"], tablefmt="fancy_grid"))

    except Exception as e:
        print(f"Error printing results \n{e}")


### How `main` works:

This function serves as the primary control center for running the frequent itemset mining analysis. It orchestrates the entire process by:

1.  **Presenting Store Options:** It displays a list of available stores with corresponding transaction and itemset files.
2.  **Getting User Input:** It prompts the user to select a store and enter the desired minimum support and confidence thresholds (as percentages).
3.  **Loading Data:** It calls `load_transaction_data` to load the selected transaction and itemset files.
4.  **Preparing Data for Algorithms:** It uses `prepare_transaction_dataframe` to convert the raw transactions into a format suitable for the Apriori and FP-Growth algorithms.
5.  **Running Algorithms:** It executes the `brute_force_algorithm`, `apriori_algorithm`, and `fpgrowth_algorithm` with the loaded data and user-defined thresholds.
6.  **Displaying Results:** For each algorithm, it uses `print_results` to display the found frequent itemsets and association rules in a clear, tabular format.
7.  **Comparing Performance:** It tracks and displays the execution time for each algorithm.
8.  **Identifying Fastest Algorithm:** Finally, it determines and announces which algorithm completed the task the fastest.

In essence, the `main` function is the entry point that brings all the other functions together to perform the itemset mining analysis based on user interaction.

In [ ]:
def main():
    """
    Main entry point to compare Brute Force, Apriori, and FP-Growth algorithms on transaction data.

    This function allows the user to select a store, load its transaction data, set minimum
    support and confidence thresholds, and run three frequent itemset mining algorithms.
    It prints the frequent itemsets, association rules, execution times, and identifies
    the fastest algorithm.

    Args:
        None: All inputs are collected via user prompts.

    Returns:
        None: This function only prints results; it does not return any value.

    Notes:
        - Uses `load_transaction_data` to read transactions and item mappings from CSV files.
        - Converts transactions into a one-hot encoded DataFrame for mlxtend algorithms.
        - Runs `brute_force_algorithm`, `apriori_algorithm`, and `fpgrowth_algorithm`.
        - Prints results for each algorithm using `print_results`.
        - Displays execution times and highlights the fastest algorithm.
    """

    stores = {
        "1": ("Amazon", "Amazon_Transaction.csv", "Amazon_Itemset.csv"),
        "2": ("Bestbuy", "Bestbuy_Transaction.csv", "Bestbuy_Itemset.csv"),
        "3": ("Shoprite", "Shoprite_Transaction.csv", "Shoprite_Itemset.csv"),
        "4": ("K-mart", "K_mart_Transaction.csv", "K_mart_Itemset.csv"),
        "5": ("Nike", "Nike_Transaction.csv", "Nike_Itemset.csv")
    }

    print("\nWelcome to ruleFinder!")
    print("\nAvailable stores:")
    for key, (store_name, _, _) in stores.items():
        print(f"{key}. {store_name}")

    # --- Store selection ---
    while True:
        choice = input("Enter the number of the store you want to analyze: ").strip()
        if choice in stores:
            store_name, transaction_file, itemset_file = stores[choice]
            break
        else:
            print("Invalid choice. Please enter a number between 1 and 5.")

    # --- Load data ---
    transaction_file = "../data/" + transaction_file
    itemset_file = "../data/" + itemset_file
    transactions, item_map = load_transaction_data(transaction_file, itemset_file)
    if not transactions:
        print("No valid transactions found. Please check your CSV files.")
        return

    # --- User input for thresholds ---
    while True:
        try:
            min_support = float(input("Enter the minimum support (as % between 1 and 100): "))
            if 1 <= min_support <= 100:
                min_support /= 100
                break
            print("Please enter a value between 1 and 100.")
        except ValueError:
            print("Invalid input. Enter a numeric value.")

    while True:
        try:
            min_confidence = float(input("Enter the minimum confidence (as % between 1 and 100): "))
            if 1 <= min_confidence <= 100:
                min_confidence /= 100
                break
            print("Please enter a value between 1 and 100.")
        except ValueError:
            print("Invalid input. Enter a numeric value.")

    # --- Prepare DataFrame for mlxtend algorithms ---
    transaction_df = prepare_transaction_dataframe(transactions)

    print(f"\nAnalyzing store: {store_name}\n{'-'*60}")

    # --- Run algorithms ---
    bf_itemsets, bf_rules, bf_time = brute_force_algorithm(transactions, min_support, min_confidence)
    apr_itemsets, apr_rules, apr_time = apriori_algorithm(transaction_df, min_support, min_confidence)
    fp_itemsets, fp_rules, fp_time = fpgrowth_algorithm(transaction_df, min_support, min_confidence)


    results_table = [
        ["Brute Force", f"{bf_time:.4f}"],
        ["Apriori", f"{apr_time:.4f}"],
        ["FP-Growth", f"{fp_time:.4f}"]
    ]

    print("\nExecution Times (seconds):")
    print(tabulate(results_table, headers=["Algorithm", "Time (s)"], tablefmt="grid"))

    # --- Determine fastest algorithm ---
    times = {
        "Brute Force": bf_time,
        "Apriori": apr_time,
        "FP-Growth": fp_time
    }
    fastest = min(times, key=times.get)
    print(f"\n The fastest algorithm is: {fastest} ({times[fastest]:.4f} seconds)\n")


if __name__ == "__main__":
    main()



Welcome to ruleFinder!

Available stores:
1. Amazon
2. Bestbuy
3. Shoprite
4. K-mart
5. Nike
Enter the number of the store you want to analyze: 1
Enter the minimum support (as % between 1 and 100): 59
Enter the minimum confidence (as % between 1 and 100): 59

Analyzing store: Amazon
------------------------------------------------------------

Running Brute Force Algorithm...
Brute Force completed in 0.0038 seconds.


=== Brute Force Results ===

Frequent Itemsets:
╒═════════════════════════════════════════╤═══════════╕
│ Itemset                                 │ Support   │
╞═════════════════════════════════════════╪═══════════╡
│ Android Programming: The Big Nerd Ranch │ 65.00%    │
├─────────────────────────────────────────┼───────────┤
│ Java For Dummies                        │ 65.00%    │
╘═════════════════════════════════════════╧═══════════╛

Association Rules:
╒════════════════════════════╤══════════════╤═══════════╕
│ Rule                       │ Confidence   │ Support   │
╞